## Main Evaluation: Seeding Strategies in Search-Based Unit Test Generation for Python

Provides the empirical evaluation for the bachelor's thesis: Seeding Strategies in Search-Based Unit Test Generation for Python

In [28]:
# Do all necessary imports here
import itertools as it
import statistics

from bisect import bisect_left
from pathlib import Path
from typing import List, Tuple, Optional, Dict

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylatex
import scipy.stats as ss
import seaborn as sns
import glob

from pandas import Categorical

### Useful Functions for Evaluation

Implement the Vargha and Delaney (Â12) effect size statistics,
taken from a [GitHub Gist](https://gist.github.com/jacksonpradolima/f9b19d65b7f16603c837024d5f8c8a65).

In [29]:
def vd_a(treatment: List[float], control: List[float]) -> Tuple[float, str]:
    """Compute Vargha and Delaney A index

    A. Vargha and H. D. Delaney.  A critique and improvement of the CL common language
    effect size statistics of McGraw and Wong.  Journal of Educational and Behavioral
    Statistics, 25(2):101-132, 2000.

    The formula to compute A has been transformed to minimise accuracy errors, see
    https://mtorchiano.wordpress.com/2014/05/19/effect-size-of-r-precision/

    :param treatment: a list of numbers
    :param control: a list of numbers
    :return: the value estimate and the magnitude
    """
    m = len(treatment)
    n = len(control)

    #if m != n:
    #    raise ValueError("Parameter lists must have equal lengths")

    r = ss.rankdata(treatment + control)
    r1 = sum(r[0:m])

    # Compute the measure
    # A = (r1/m - (m+1)/2)/n  # formula (14) in Vargha and Delaney, 2000
    A = (2 * r1 - m * (m + 1)) / (2 * n * m)  # equivalent formula with better accuracy

    levels = [0.147, 0.33, 0.474]
    magnitudes = ["negligible", "small", "medium", "large"]
    scaled_A = (A - 0.5) * 2

    magnitude = magnitudes[bisect_left(levels, abs(scaled_A))]
    estimate = A

    return estimate, magnitude

Implement helper functions for calculating the Vargha and Delaney (Â12) effect size statistics and the Mann Whitney U Test for the different scenarios.

In [55]:
def calc_a12(projectname: str, strategy1: str, strategy2: str, cov_values: pd.DataFrame):
    treatment_row = cov_values.loc[(cov_values['ConfigurationId'] == strategy1) & (cov_values['Project'] == projectname)]
    control_row = cov_values.loc[(cov_values['ConfigurationId'] == strategy2) & (cov_values['Project'] == projectname)]
    treatment = treatment_row.iloc[0]['Coverage Values']
    control = control_row.iloc[0]['Coverage Values']
    return vd_a(treatment, control)

In [144]:
def calc_mwu2(projectname: str, strategy1: str, strategy2: str, vals: pd.DataFrame):
    alpha = 0.05
    p_vals = []

    for _, row in cov_values_by_module.iterrows():
        if row[config] == strategy1 and row[project] == projectname:
            l_s2 = vals.loc[(vals[config] == strategy2) & (vals[project] == projectname) & (vals[cut] == row[cut])]
            try:               
                mwu = ss.mannwhitneyu(row['Coverage Values'], l_s2.iloc[0]['Coverage Values'], alternative='greater')
                p_vals.append(mwu.pvalue)
            except IndexError:
                pass # if for one config the run failed, nothing can be added
            except ValueError:
                p_vals.append(1) # if all values for both strategies are equal

    count_sig = 0
    for val in p_vals:
        if val < alpha:
            count_sig += 1
            
    return float(count_sig) / len(p_vals)

### Load Data From CSV Files

In [33]:
def reset_orig_dir():
    os.chdir('/home/l_pc1-l/ba/own_stuff/ba-thesis/evaluation/eval_env/notebooks')

In [34]:
# Combines all csv files in the current directory and recursively to one csv file.
#adjust the below path on your machine
os.chdir("/home/l_pc1-l/ba/own_stuff/ba-thesis/evaluation/results/main_run/data")
extension = 'csv'
all_filenames = [i for i in glob.glob('./**/*.{}'.format(extension), recursive=True)]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "../results.csv", index=False, encoding='utf-8-sig')

In [35]:
reset_orig_dir()
# The names of the columns we are interested in
cut = "TargetModule"
project_name = "ProjectName"
config = "ConfigurationId"
coverage = "Coverage"

# How often every CUT was executed
runs = 30
os.getcwd()
# Adjust the following paths on your system if you want to rerun this sheet!
PAPER_EXPORT_PATH = Path("/home/l_pc1-l/ba/own_stuff/ba-thesis/evaluation")

results = pd.read_csv(Path("../..") / "results" / "main_run" / "results.csv")
project_information = pd.read_csv(Path("../..") / "results" / "main_run" / "projects.csv")

In [36]:
number_cuts = len(set(results[cut]))
print(f"I tested {number_cuts} unique classes, each being executed {runs} times per configuration")

I tested 106 unique classes, each being executed 30 times per configuration


In [37]:
f_config_names = list(set(results[config]))
config_names = [n for n in f_config_names if type(n) is str]
config_names.sort()
print("I used {} configurations, namely:\n - {}".format(
    len(config_names), "\n - ".join(config_names)
))

I used 5 configurations, namely:
 - All
 - Baseline
 - Initial
 - Static
 - Static_and_Dynamic


In [38]:
# Add a column containing the project name to the evaluation results.
projects = []
for _, row in results.iterrows():
    projects.append(row[cut].split(".")[0])

results.insert(1, "Project", projects, True)
project = "Project"

### Creating different datasets for the different tables

The results obtained from the Pynguin runs need to be adapted to my needs. I create different datasets which I use later to create tables.

In [63]:
table_data = results.groupby([config, project], as_index=False).agg(
    {
        coverage: "mean",
    }
)

cov_values = results.groupby([config, project])[coverage].apply(list).reset_index(name='Coverage Values')
cov_values_by_module = results.groupby([config, project, cut])[coverage].apply(list).reset_index(name='Coverage Values')


merged = pd.merge(table_data, table_data, on=project)
static_baseline = merged.loc[(merged['ConfigurationId_x'] == 'Baseline') & (merged['ConfigurationId_y'] == 'Static')]
static_baseline.insert(2, "# Modules", [1,6,2,6,9,13,34,6,23,4], True)

static_dynamic_baseline = merged.loc[(merged['ConfigurationId_x'] == 'Baseline') & (merged['ConfigurationId_y'] == 'Static_and_Dynamic')]
static_dynamic_baseline.insert(2, "# Modules", [1,6,2,6,9,13,34,6,23,4], True)

initial_baseline = merged.loc[(merged['ConfigurationId_x'] == 'Baseline') & (merged['ConfigurationId_y'] == 'Initial')]
initial_baseline.insert(2, "# Modules", [1,6,2,6,9,13,34,6,23,4], True)
initial_baseline.insert(4, "Avg Coll. Testcases", [0.0,3.0,0.0,0.5,2.5,6.9,10.5,18.0,0.0,0.0], True)

static_dynamic_static = merged.loc[(merged['ConfigurationId_x'] == 'Static') & (merged['ConfigurationId_y'] == 'Static_and_Dynamic')]
static_dynamic_static.insert(2, "# Modules", [1,6,2,6,9,13,34,6,23,4], True)

all_baseline = merged.loc[(merged['ConfigurationId_x'] == 'Baseline') & (merged['ConfigurationId_y'] == 'All')]
all_baseline.insert(2, "# Modules", [1,6,2,6,9,13,34,6,23,4], True)

## Tables for the different strategies

In the following I create Latex tables to visualize my answers to the research questions.

In [165]:
# Table for evaluating the static constant seeding against the baseline
table = pylatex.Table()
table.append(pylatex.NoEscape(r'\centering'))
table.append(pylatex.NoEscape(r"\resizebox{\columnwidth}{!}{"))
tabular = pylatex.Tabular('|l|c|c|c|c|c|c|', booktabs=True)
tabular.add_row([
    "Project",
    "# Modules",
    "Baseline",
    "Static",
    pylatex.NoEscape(r"Â\textsubscript{12}"),
    "% Improved",
    "% Degraded"
])

tabular.add_hline()
for _, row in static_baseline.iterrows():
    tabular.add_row([
        row[project],
        row['# Modules'],
        "{:.2f}".format(row['Coverage_x']),
        "{:.2f}".format(row['Coverage_y']),
        "{:.2f}".format(calc_a12(row[project], "Static", "Baseline", cov_values)[0]),
        "{:.2f}".format(calc_mwu2(row[project], "Static", "Baseline", cov_values_by_module)),
        "{:.2f}".format(calc_mwu2(row[project], "Baseline", "Static", cov_values_by_module))
    ])
tabular.add_hline()
sum_a12 = 0.0
sum_mwu1 = 0.0
sum_mwu2 = 0.0
for _, row in static_baseline.iterrows():
    sum_a12 += float("{:.2f}".format(calc_a12(row[project], "Static", "Baseline", cov_values)[0]))
    sum_mwu1 += float("{:.2f}".format(calc_mwu2(row[project], "Static", "Baseline", cov_values_by_module)))
    sum_mwu2 += float("{:.2f}".format(calc_mwu2(row[project], "Baseline", "Static", cov_values_by_module)))
tabular.add_row([
    "Average",
    "{:.1f}".format(static_baseline['# Modules'].mean()),
    "{:.2f}".format(static_baseline['Coverage_x'].mean()),
    "{:.2f}".format(static_baseline['Coverage_y'].mean()),
    "{:.2f}".format(sum_a12 / 10),
    "{:.2f}".format(sum_mwu1 / 10),
    "{:.2f}".format(sum_mwu2 / 10)
])
    
tabular.add_hline()
table.append(tabular)
table.append(pylatex.NoEscape(r'}'))
table.add_caption(pylatex.NoEscape(r"Shows for each project the average coverage achieved with the" +
                                   r" two strategies Baseline (P\textsubscript{Constant} = 0) and" +
                                   r" Static Constant Seeding (P\textsubscript{Constant} = 0.4)." +
                                   r" The Â\textsubscript{12} value is calculated by averaging all runs" + 
                                   r" of all classes per project. The last two columns report the percentage" +
                                   r" of classes for which there is a significant (p-value lower than 0.05)" +
                                   r" improvement/degradation of the second configuration compared to the first."))
label = pylatex.Label("tabStaticBase")
table.append(label)

#adjust this path if you want to store the table on your machine
with open ("../../../Thesis/chapters/evaluation_tables/base_static_table.tex", "w") as file:
    file.write(table.dumps())
print(table.dumps())

\begin{table}%
\centering%
\resizebox{\columnwidth}{!}{%
\begin{tabular}{@{}|l|c|c|c|c|c|c|@{}}%
\toprule%
Project&\# Modules&Baseline&Static&Â\textsubscript{12}&\% Improved&\% Degraded\\%
\midrule%
apimd&1&0.26&0.39&0.85&1.00&0.00\\%
async\_btree&6&0.30&0.31&0.51&0.17&0.00\\%
codetiming&2&0.92&0.92&0.53&0.00&0.00\\%
docstring\_parser&6&0.52&0.56&0.55&0.33&0.17\\%
flutes&9&0.39&0.39&0.50&0.00&0.00\\%
flutils&13&0.45&0.47&0.52&0.23&0.00\\%
mimesis&34&0.88&0.88&0.49&0.06&0.08\\%
pypara&6&0.39&0.39&0.50&0.00&0.00\\%
pytutils&23&0.60&0.60&0.50&0.09&0.00\\%
string\_utils&4&0.86&0.88&0.54&0.50&0.00\\%
\midrule%
Average&10.4&0.56&0.58&0.55&0.24&0.03\\%
\midrule\bottomrule%
%
\end{tabular}%
}%
\caption{Shows for each project the average coverage achieved with the two strategies Baseline (P\textsubscript{Constant} = 0) and Static Constant Seeding (P\textsubscript{Constant} = 0.4). The Â\textsubscript{12} value is calculated by averaging all runs of all classes per project. The last two columns 

In [166]:
#table for evaluating static and dynamic constant seeding against Baseline
table = pylatex.Table(position="H")
table.append(pylatex.NoEscape(r'\centering'))
table.append(pylatex.NoEscape(r"\resizebox{\columnwidth}{!}{"))
tabular = pylatex.Tabular('|l|c|c|c|c|c|c|', booktabs=True)
tabular.add_row([
    "Project",
    "# Modules",
    "Baseline",
    "Static + Dynamic",
    pylatex.NoEscape(r"Â\textsubscript{12}"),
    "% Improved",
    "% Degraded"
])

tabular.add_hline()
for _, row in static_dynamic_baseline.iterrows():
    tabular.add_row([
        row[project],
        row['# Modules'],
        "{:.2f}".format(row['Coverage_x']),
        "{:.2f}".format(row['Coverage_y']),
        "{:.2f}".format(calc_a12(row[project], "Static_and_Dynamic", "Baseline", cov_values)[0]),
        "{:.2f}".format(calc_mwu2(row[project], "Static_and_Dynamic", "Baseline", cov_values_by_module)),
        "{:.2f}".format(calc_mwu2(row[project], "Baseline", "Static_and_Dynamic", cov_values_by_module))
    ])
    
tabular.add_hline()
sum_a12 = 0.0
sum_mwu1 = 0.0
sum_mwu2 = 0.0
for _, row in static_dynamic_baseline.iterrows():
    sum_a12 += float("{:.2f}".format(calc_a12(row[project], "Static_and_Dynamic", "Baseline", cov_values)[0]))
    sum_mwu1 += float("{:.2f}".format(calc_mwu2(row[project], "Static_and_Dynamic", "Baseline", cov_values_by_module)))
    sum_mwu2 += float("{:.2f}".format(calc_mwu2(row[project], "Baseline", "Static_and_Dynamic", cov_values_by_module)))
tabular.add_row([
    "Average",
    "{:.1f}".format(static_dynamic_baseline['# Modules'].mean()),
    "{:.2f}".format(static_dynamic_baseline['Coverage_x'].mean()),
    "{:.2f}".format(static_dynamic_baseline['Coverage_y'].mean()),
    "{:.2f}".format(sum_a12 / 10),
    "{:.2f}".format(sum_mwu1 / 10),
    "{:.2f}".format(sum_mwu2 / 10)
])

tabular.add_hline()
table.append(tabular)
table.append(pylatex.NoEscape(r'}'))
table.add_caption(pylatex.NoEscape(r"Shows for each project the average coverage achieved with the" +
                                   r" two strategies Baseline (P\textsubscript{Constant} = 0," + 
                                   r" P\textsubscript{Dynamic} = 0) and" +
                                   r" Static + Dynamic Constant Seeding (P\textsubscript{Constant} = 0.2," +
                                   r" P\textsubscript{Dynamic} = 0.6)." +
                                   r" The Â\textsubscript{12} value is calculated by averaging all runs" + 
                                   r" of all classes per project. The last two columns report the percentage" +
                                   r" of classes for which there is a significant (p-value lower than 0.05)" +
                                   r" improvement/degradation of the second configuration compared to the first."))
label = pylatex.Label("tabStaticDynamicBase")
table.append(label)

#adjust this path if you want to store the table on your machine
with open ("../../../Thesis/chapters/evaluation_tables/base_static+dynamic_table.tex", "w") as file:
    file.write(table.dumps())
print(table.dumps())

\begin{table}[H]%
\centering%
\resizebox{\columnwidth}{!}{%
\begin{tabular}{@{}|l|c|c|c|c|c|c|@{}}%
\toprule%
Project&\# Modules&Baseline&Static + Dynamic&Â\textsubscript{12}&\% Improved&\% Degraded\\%
\midrule%
apimd&1&0.26&0.31&0.58&0.00&0.00\\%
async\_btree&6&0.30&0.30&0.50&0.00&0.00\\%
codetiming&2&0.92&0.92&0.53&0.50&0.00\\%
docstring\_parser&6&0.52&0.56&0.54&0.33&0.00\\%
flutes&9&0.39&0.40&0.51&0.00&0.00\\%
flutils&13&0.45&0.43&0.48&0.27&0.00\\%
mimesis&34&0.88&0.88&0.49&0.06&0.00\\%
pypara&6&0.39&0.39&0.51&0.00&0.00\\%
pytutils&23&0.60&0.61&0.50&0.04&0.00\\%
string\_utils&4&0.86&0.87&0.53&0.50&0.00\\%
\midrule%
Average&10.4&0.56&0.57&0.52&0.17&0.00\\%
\midrule\bottomrule%
%
\end{tabular}%
}%
\caption{Shows for each project the average coverage achieved with the two strategies Baseline (P\textsubscript{Constant} = 0, P\textsubscript{Dynamic} = 0) and Static + Dynamic Constant Seeding (P\textsubscript{Constant} = 0.2, P\textsubscript{Dynamic} = 0.6). The Â\textsubscript{12} value 

In [167]:
#table for evaluating static and dynamic constant seeding against static constant seeding
table = pylatex.Table(position="H")
table.append(pylatex.NoEscape(r'\centering'))
table.append(pylatex.NoEscape(r"\resizebox{\columnwidth}{!}{"))
tabular = pylatex.Tabular('|l|c|c|c|c|c|c|', booktabs=True)
tabular.add_row([
    "Project",
    "# Modules",
    "Static",
    "Static + Dynamic",
    pylatex.NoEscape(r"Â\textsubscript{12}"),
    "% Improved",
    "% Degraded"
])

tabular.add_hline()
for _, row in static_dynamic_static.iterrows():
    tabular.add_row([
        row[project],
        row['# Modules'],
        "{:.2f}".format(row['Coverage_x']),
        "{:.2f}".format(row['Coverage_y']),
        "{:.2f}".format(calc_a12(row[project], "Static_and_Dynamic", "Static", cov_values)[0]),
        "{:.2f}".format(calc_mwu2(row[project], "Static_and_Dynamic", "Static", cov_values_by_module)),
        "{:.2f}".format(calc_mwu2(row[project], "Static", "Static_and_Dynamic", cov_values_by_module))
    ])
    
tabular.add_hline()
sum_a12 = 0.0
sum_mwu1 = 0.0
sum_mwu2 = 0.0
for _, row in static_dynamic_static.iterrows():
    sum_a12 += float("{:.2f}".format(calc_a12(row[project], "Static_and_Dynamic", "Static", cov_values)[0]))
    sum_mwu1 += float("{:.2f}".format(calc_mwu2(row[project], "Static_and_Dynamic", "Static", cov_values_by_module)))
    sum_mwu2 += float("{:.2f}".format(calc_mwu2(row[project], "Static", "Static_and_Dynamic", cov_values_by_module)))
tabular.add_row([
    "Average",
    "{:.1f}".format(static_dynamic_static['# Modules'].mean()),
    "{:.2f}".format(static_dynamic_static['Coverage_x'].mean()),
    "{:.2f}".format(static_dynamic_static['Coverage_y'].mean()),
    "{:.2f}".format(sum_a12 / 10),
    "{:.2f}".format(sum_mwu1 / 10),
    "{:.2f}".format(sum_mwu2 / 10)
])


tabular.add_hline()
table.append(tabular)
table.append(pylatex.NoEscape(r'}'))
table.add_caption(pylatex.NoEscape(r"Shows for each project the average coverage achieved with the" +
                                   r" two strategies Static Constant Seeding (P\textsubscript{Constant} = 0.4) and" +
                                   r" Static + Dynamic Constant Seeding (P\textsubscript{Constant} = 0.2," +
                                   r" P\textsubscript{Dynamic} = 0.6)." +
                                   r" The Â\textsubscript{12} value is calculated by averaging all runs" + 
                                   r" of all classes per project. The last two columns report the percentage" +
                                   r" of classes for which there is a significant (p-value lower than 0.05)" +
                                   r" improvement/degradation of the second configuration compared to the first."))
label = pylatex.Label("tabStaticDynamicStatic")
table.append(label)

#adjust this path if you want to store the table on your machine
with open ("../../../Thesis/chapters/evaluation_tables/static_static+dynamic_table.tex", "w") as file:
    file.write(table.dumps())
print(table.dumps())

\begin{table}[H]%
\centering%
\resizebox{\columnwidth}{!}{%
\begin{tabular}{@{}|l|c|c|c|c|c|c|@{}}%
\toprule%
Project&\# Modules&Static&Static + Dynamic&Â\textsubscript{12}&\% Improved&\% Degraded\\%
\midrule%
apimd&1&0.39&0.31&0.28&0.00&1.00\\%
async\_btree&6&0.31&0.30&0.49&0.00&0.17\\%
codetiming&2&0.92&0.92&0.51&0.00&0.00\\%
docstring\_parser&6&0.56&0.56&0.49&0.00&0.00\\%
flutes&9&0.39&0.40&0.50&0.00&0.14\\%
flutils&13&0.47&0.43&0.46&0.09&0.00\\%
mimesis&34&0.88&0.88&0.50&0.08&0.03\\%
pypara&6&0.39&0.39&0.51&0.00&0.00\\%
pytutils&23&0.60&0.61&0.50&0.04&0.00\\%
string\_utils&4&0.88&0.87&0.48&0.00&0.25\\%
\midrule%
Average&10.4&0.58&0.57&0.47&0.02&0.16\\%
\midrule\bottomrule%
%
\end{tabular}%
}%
\caption{Shows for each project the average coverage achieved with the two strategies Static Constant Seeding (P\textsubscript{Constant} = 0.4) and Static + Dynamic Constant Seeding (P\textsubscript{Constant} = 0.2, P\textsubscript{Dynamic} = 0.6). The Â\textsubscript{12} value is calculated b

In [168]:
#table for evaluating initial population seeding against Baseline
table = pylatex.Table(position="H")
table.append(pylatex.NoEscape(r'\centering'))
table.append(pylatex.NoEscape(r"\resizebox{\columnwidth}{!}{"))
tabular = pylatex.Tabular('|l|c|c|c|c|c|c|c|', booktabs=True)
tabular.add_row([
    "Project",
    "# Modules",
    "Baseline",
    "Initial",
    "Avg Coll. Testcases",
    pylatex.NoEscape(r"Â\textsubscript{12}"),
    "% Improved",
    "% Degraded"
])

tabular.add_hline()
for _, row in initial_baseline.iterrows():
    tabular.add_row([
        row[project],
        row['# Modules'],
        "{:.2f}".format(row['Coverage_x']),
        "{:.2f}".format(row['Coverage_y']),
        row["Avg Coll. Testcases"],
        "{:.2f}".format(calc_a12(row[project], "Initial", "Baseline", cov_values)[0]),
        "{:.2f}".format(calc_mwu2(row[project], "Initial", "Baseline", cov_values_by_module)),
        "{:.2f}".format(calc_mwu2(row[project], "Baseline", "Initial", cov_values_by_module))
    ])

tabular.add_hline()
sum_a12 = 0.0
sum_mwu1 = 0.0
sum_mwu2 = 0.0
for _, row in initial_baseline.iterrows():
    sum_a12 += float("{:.2f}".format(calc_a12(row[project], "Initial", "Baseline", cov_values)[0]))
    sum_mwu1 += float("{:.2f}".format(calc_mwu2(row[project], "Initial", "Baseline", cov_values_by_module)))
    sum_mwu2 += float("{:.2f}".format(calc_mwu2(row[project], "Baseline", "Initial", cov_values_by_module)))
tabular.add_row([
    "Average",
    "{:.1f}".format(initial_baseline['# Modules'].mean()),
    "{:.2f}".format(initial_baseline['Coverage_x'].mean()),
    "{:.2f}".format(initial_baseline['Coverage_y'].mean()),
    "{:.1f}".format(initial_baseline['Avg Coll. Testcases'].mean()),
    "{:.2f}".format(sum_a12 / 10),
    "{:.2f}".format(sum_mwu1 / 10),
    "{:.2f}".format(sum_mwu2 / 10)
])

    
tabular.add_hline()
table.append(tabular)
table.append(pylatex.NoEscape(r'}'))
table.add_caption(pylatex.NoEscape(r"Shows for each project the average coverage achieved with the" +
                                   r" two strategies Baseline (P\textsubscript{Clone} = 0," +
                                   r" N\textsubscript{Mutations} = 0) and" +
                                   r" Initial Population Seeding (P\textsubscript{Clone} = 0.3," +
                                   r" N\textsubscript{Mutations} = 8)." +
                                   r" The Â\textsubscript{12} value is calculated by averaging all runs" + 
                                   r" of all classes per project. The last two columns report the percentage" +
                                   r" of classes for which there is a significant (p-value lower than 0.05)" +
                                   r" improvement/degradation of the second configuration compared to the first."))
label = pylatex.Label("tabInitialBase")
table.append(label)

#adjust this path if you want to store the table on your machine
with open ("../../../Thesis/chapters/evaluation_tables/base_initial_table.tex", "w") as file:
    file.write(table.dumps())
print(table.dumps())

\begin{table}[H]%
\centering%
\resizebox{\columnwidth}{!}{%
\begin{tabular}{@{}|l|c|c|c|c|c|c|c|@{}}%
\toprule%
Project&\# Modules&Baseline&Initial&Avg Coll. Testcases&Â\textsubscript{12}&\% Improved&\% Degraded\\%
\midrule%
apimd&1&0.26&0.26&0.0&0.41&0.00&0.00\\%
async\_btree&6&0.30&0.33&3.0&0.54&0.00&0.20\\%
codetiming&2&0.92&0.92&0.0&0.51&0.00&0.00\\%
docstring\_parser&6&0.52&0.52&0.5&0.49&0.00&0.17\\%
flutes&9&0.39&0.43&2.5&0.52&0.00&0.00\\%
flutils&13&0.45&0.50&6.9&0.55&0.20&0.00\\%
mimesis&34&0.88&0.88&10.5&0.48&0.00&0.03\\%
pypara&6&0.39&0.39&18.0&0.51&0.00&0.00\\%
pytutils&23&0.60&0.60&0.0&0.50&0.00&0.00\\%
string\_utils&4&0.86&0.87&0.0&0.50&0.00&0.00\\%
\midrule%
Average&10.4&0.56&0.57&4.1&0.50&0.02&0.04\\%
\midrule\bottomrule%
%
\end{tabular}%
}%
\caption{Shows for each project the average coverage achieved with the two strategies Baseline (P\textsubscript{Clone} = 0, N\textsubscript{Mutations} = 0) and Initial Population Seeding (P\textsubscript{Clone} = 0.3, N\textsubscript

In [164]:
#table for evaluating all seeding strategies combined against Baseline
table = pylatex.Table(position="H")
table.append(pylatex.NoEscape(r'\centering'))
table.append(pylatex.NoEscape(r"\resizebox{\columnwidth}{!}{"))
tabular = pylatex.Tabular('|l|c|c|c|c|c|c|', booktabs=True)
tabular.add_row([
    "Project",
    "# Modules",
    "Baseline",
    "Static + Dynamic + Initial",
    pylatex.NoEscape(r"Â\textsubscript{12}"),
    "% Improved",
    "% Degraded"
])

tabular.add_hline()
for _, row in all_baseline.iterrows():
    tabular.add_row([
        row[project],
        row['# Modules'],
        "{:.2f}".format(row['Coverage_x']),
        "{:.2f}".format(row['Coverage_y']),
        "{:.2f}".format(calc_a12(row[project], "All", "Baseline", cov_values)[0]),
        "{:.2f}".format(calc_mwu2(row[project], "All", "Baseline", cov_values_by_module)),
        "{:.2f}".format(calc_mwu2(row[project], "Baseline", "All", cov_values_by_module))
    ])

tabular.add_hline()
sum_a12 = 0.0
sum_mwu1 = 0.0
sum_mwu2 = 0.0
for _, row in all_baseline.iterrows():
    sum_a12 += float("{:.2f}".format(calc_a12(row[project], "All", "Baseline", cov_values)[0]))
    sum_mwu1 += float("{:.2f}".format(calc_mwu2(row[project], "All", "Baseline", cov_values_by_module)))
    sum_mwu2 += float("{:.2f}".format(calc_mwu2(row[project], "Baseline", "All", cov_values_by_module)))
tabular.add_row([
    "Average",
    "{:.1f}".format(all_baseline['# Modules'].mean()),
    "{:.2f}".format(all_baseline['Coverage_x'].mean()),
    "{:.2f}".format(all_baseline['Coverage_y'].mean()),
    "{:.2f}".format(sum_a12 / 10),
    "{:.2f}".format(sum_mwu1 / 10),
    "{:.2f}".format(sum_mwu2 / 10)
])

    
tabular.add_hline()
table.append(tabular)
table.append(pylatex.NoEscape(r'}'))
table.add_caption(pylatex.NoEscape(r"Shows for each project the average coverage achieved with the" +
                                   r" two strategies Baseline (no seeding at all) and" +
                                   r" All (P\textsubscript{Constant} = 0.2," +
                                   r" P\textsubscript{Dynamic} = 0.6, P\textsubscript{Clone} = 0.3," +
                                   r" N\textsubscript{Mutations} = 8)." +
                                   r" The Â\textsubscript{12} value is calculated by averaging all runs" + 
                                   r" of all classes per project. The last two columns report the percentage" +
                                   r" of classes for which there is a significant (p-value lower than 0.05)" +
                                   r" improvement/degradation of the second configuration compared to the first."))
label = pylatex.Label("tabAllBase")
table.append(label)

#adjust this path if you want to store the table on your machine
with open ("../../../Thesis/chapters/evaluation_tables/base_all_table.tex", "w") as file:
    file.write(table.dumps())
print(table.dumps())

\begin{table}[H]%
\centering%
\resizebox{\columnwidth}{!}{%
\begin{tabular}{@{}|l|c|c|c|c|c|c|@{}}%
\toprule%
Project&\# Modules&Baseline&Static + Dynamic + Initial&Â\textsubscript{12}&\% Improved&\% Degraded\\%
\midrule%
apimd&1&0.26&0.28&0.56&0.00&0.00\\%
async\_btree&6&0.30&0.34&0.56&0.20&0.00\\%
codetiming&2&0.92&0.92&0.52&0.00&0.00\\%
docstring\_parser&6&0.52&0.56&0.54&0.33&0.00\\%
flutes&9&0.39&0.43&0.53&0.00&0.00\\%
flutils&13&0.45&0.49&0.53&0.50&0.12\\%
mimesis&34&0.88&0.88&0.48&0.03&0.00\\%
pypara&6&0.39&0.39&0.53&0.40&0.00\\%
pytutils&23&0.60&0.61&0.51&0.09&0.00\\%
string\_utils&4&0.86&0.87&0.53&0.50&0.00\\%
\midrule%
Average&10.4&0.56&0.58&0.53&0.20&0.01\\%
\midrule\bottomrule%
%
\end{tabular}%
}%
\caption{Shows for each project the average coverage achieved with the two strategies Baseline (no seeding at all) and All (P\textsubscript{Constant} = 0.2, P\textsubscript{Dynamic} = 0.6, P\textsubscript{Clone} = 0.3, N\textsubscript{Mutations} = 8). The Â\textsubscript{12} value 